# Dummy Net

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.data import init_model_logging
import tensorflow as tf
import numpy as np
import pandas as pd
import os

## Animals Data

In [4]:
# [fur, extinction, weight] 
animals = np.array([[1.0, 0.7, 0.9], # lion
                    [0.8, 0.8, 1.0], # lion
                    [0.1, 0.3, 0.2], # seal
                    [0.0, 0.2, 0.1]  # seal
                   ], dtype=np.float32)

In [5]:
animal_labels = np.array(
                [[ 1.0],  # lion
                 [ 1.0],  # lion
                 [-1.0],  # seal
                 [-1.0]   # seal
                ], dtype=np.float32)

## Net Without Training 

### Build Graph

In [4]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    
    w1 = tf.constant(np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32), name='w1_const')
    h = tf.matmul(x, w1)

    w2 = tf.constant(np.array([[1], [1]], dtype=np.float32), name='w2_const')
    y = tf.matmul(h, w2)

### Run Session

In [5]:
session = tf.Session(graph=graph)

In [6]:
prediction, = session.run([y], feed_dict={x: animals})
print('Prediction on animals:')
print(prediction)

Prediction on animals:
[[ 22.79999924]
 [ 24.60000038]
 [  6.        ]
 [  3.29999995]]


In [7]:
session.close()

## Net With Loss

### Build Graph

In [8]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.constant(np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32), name='w1_const')
    h = tf.matmul(x, w1)

    w2 = tf.constant(np.array([[1], [1]], dtype=np.float32), name='w2_const')
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)

### Run Session

In [9]:
session = tf.Session(graph=graph)

In [10]:
loss_on_animal_data, = session.run([loss], feed_dict={x: animals, label: animal_labels})
print('Loss: {0:.2f}'.format(loss_on_animal_data))

Loss: 274.92


In [11]:
session.close()

## Net With Trainable Variables

### Build Graph

In [12]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
    h = tf.matmul(x, w1)

    w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)
    initialize_vars = tf.global_variables_initializer()

### Run Session

In [13]:
session = tf.Session(graph=graph)

In [14]:
session.run(initialize_vars)
w1_values, w2_values = session.run([w1, w2], feed_dict={x: animals, label: animal_labels})
print('w1:')
print(w1_values)
print('w2:')
print(w2_values)

w1:
[[ 0.10959081 -0.13871621]
 [ 0.05815202 -0.01618201]
 [ 0.05937244 -0.11505231]]
w2:
[[ 0.01594922]
 [-0.03459523]]


In [ ]:
session.close()

## Net With Train Step

### Build Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
    h = tf.matmul(x, w1)

    w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

    initialize_vars = tf.global_variables_initializer()

### Run Session

In [ ]:
session = tf.Session(graph=graph)
session.run(initialize_vars)

In [ ]:
for iteration in range(10001):
    _, loss_on_animal_data = session.run([train_step, loss], feed_dict={x: animals, label: animal_labels})
    if iteration % 500 == 0:
        print("Iteration {}: loss {}".format(iteration, loss_on_animal_data))

In [ ]:
prediction = session.run(y, feed_dict={x: animals})

In [ ]:
pd.DataFrame({'prediction':prediction[:,0], 'label':animal_labels[:,0]})

In [ ]:
session.close()

## Net with logging

### Function collecting data for logging


In [1]:
def variable_summaries(name, var):
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))

        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

### Build Graph

In [6]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
        label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    with tf.variable_scope('layer_1'):
        w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
        h = tf.matmul(x, w1)
        variable_summaries('w1', w1)

    with tf.variable_scope('layer_2'):
        w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
        y = tf.matmul(h, w2)
    
    with tf.name_scope('loss'):
        loss = tf.losses.mean_squared_error(label, y)
        variable_summaries('loss', loss)

    with tf.name_scope('training'):
        train_step = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss)

    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

### Init Model Logging

In [7]:
base_dir = '/tensorboard_summaries/dummy_net/'
logging_meta = init_model_logging(base_dir, 'experiment1', graph=graph, remove_existing=True)

In [8]:
logging_meta

{'model_path': '/tensorboard_summaries/dummy_net/experiment1/valid/model.ckpt',
 'saver': <tensorflow.python.training.saver.Saver at 0x7fd8abd226d8>,
 'train_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7fd8abc79898>,
 'train_writer_dir': '/tensorboard_summaries/dummy_net/experiment1/train',
 'valid_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7fd8abd22be0>,
 'valid_writer_dir': '/tensorboard_summaries/dummy_net/experiment1/valid'}

### Run Session

In [9]:
with tf.Session(graph=graph) as session:
    session.run(initialize_vars)
    
    for iteration in range(10001):
        _, loss_on_animal_data, summaries = session.run(
            [train_step, loss, merge_summaries], feed_dict={x: animals, label: animal_labels})
    
        logging_meta['train_writer'].add_summary(summaries, iteration)
        if iteration % 500 == 0:
            print("Iteration {}: loss {}".format(iteration, loss_on_animal_data))    

    logging_meta['saver'].save(session, logging_meta['model_path'])

Iteration 0: loss 1.0303211212158203
Iteration 500: loss 0.12384260445833206
Iteration 1000: loss 0.018906988203525543
Iteration 1500: loss 0.001623040297999978
Iteration 2000: loss 0.0009231154108420014
Iteration 2500: loss 0.0009058316936716437
Iteration 3000: loss 0.0009054472902789712
Iteration 3500: loss 0.0009054334950633347
Iteration 4000: loss 0.0009054337278939784
Iteration 4500: loss 0.0009054337278939784
Iteration 5000: loss 0.0009054337278939784
Iteration 5500: loss 0.0009054337278939784
Iteration 6000: loss 0.0009054337278939784
Iteration 6500: loss 0.0009054337278939784
Iteration 7000: loss 0.0009054337278939784
Iteration 7500: loss 0.0009054337278939784
Iteration 8000: loss 0.0009054337278939784
Iteration 8500: loss 0.0009054337278939784
Iteration 9000: loss 0.0009054337278939784
Iteration 9500: loss 0.0009054337278939784
Iteration 10000: loss 0.0009054337278939784
